In [ ]:
from numpy import loadtxt
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
def mag_estimation_loss(y_true, y_pred):
    
    mask = y_true[-1] 

    return (y_true - y_pred) * mask 

In [ ]:
def custom_loss2(y_true, y_pred):
    mse = tf.keras.losses.MeanSquaredError()
    mae = tf.keras.losses.MeanAbsoluteError()
    
    weights = [0.4, 0.4, 0.2]
    
    
    return weights[0] * mse(y_true, y_pred) + weights[1] * mae(y_true, y_pred) + weights[2] * mag_estimation_loss(y_true, y_pred)

In [ ]:
import keras

model = keras.models.load_model("CREIME_Model.h5", custom_objects = {'custom_loss2': custom_loss2})

In [ ]:
model.summary()

In [ ]:
## X = "path/to/data"

""" 
the data represented by the variable X needs to be loaded
It must be an array of shape (n, 512, 3) where n is the 
total number of samples in the test data
"""
y_predicted = model.predict(X)

In [ ]:
"""
calculating the predicted p-arrival time (in seconds) 
starting from the window start time
P-arrival is considered to happen at the sample where 
the predicted sequence crosses the threshold of -0.5
"""

p_predicted = (512 - np.array([np.sum((y > -0.5).astype(int)) for y in y_predicted])) / 100

In [ ]:
"""
calculating the predicted magnitude
magnitude is considered to be the arithmetic mean 
of the last 10 samples of the output sequence
"""

m_predicted = [np.mean(y[-10:]) for y in y_predicted]

In [ ]:
"""
classifying the data points as event or noise based on predicted magnitude
if predicted magnitude is less the -0.5 the waveform is classified as noise
n_predicted is an array of 0s and 1s with 0 standing for event and 1 standing for noise
"""

n_prediction = [mag < -0.5 for mag in m_predicted]